<a href="https://colab.research.google.com/github/chvn00/sistemas_embebidos/blob/main/Proy_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementación del modelo MLP entrenado en la Raspberry Pi

##Sistema embebido con inferencia en tiempo real, control de actuadores y panel web avanzado en Flask

Después de entrenar la red neuronal MLP en este cuaderno y obtener las matrices de pesos finales (W1, b1, W2, b2), el siguiente paso es implementar la inferencia directamente en una Raspberry Pi para crear una mini línea de producción inteligente.

El propósito es integrar sensores físicos, lógica de decisión basada en IA y un panel web industrial que permita monitorear todo el sistema en tiempo real.

## Descripción general del sistema

El código proporcionado para la Raspberry Pi cumple las siguientes funciones:

- Lectura de cuatro entradas principales del proceso:

- Distancia medida con el sensor ultrasónico HC-SR04.

- Intensidad de luz o reflectancia (simulada o medida con LDR).

- Temperatura medida con un DHT11 (o simulada si el sensor no está presente).

- Modo de operación seleccionado mediante un DIP switch de dos bits.

Normalización de las lecturas, usando los mismos rangos aplicados durante el entrenamiento en Colab:

- Distancia entre 4 y 20 cm transforma a un rango 0–1.

- Luz en rango 0–1.

- Temperatura entre 20 y 90 grados Celsius transforma a 0–1.

- Modo (0 a 3) se normaliza dividiendo entre 3.

Inferencia con la red neuronal MLP entrenada, cuya arquitectura es:

- 4 entradas.

- 32 neuronas en una capa oculta con activación ReLU.

- 3 salidas que representan: ACEPTADO, EN REVISIÓN, y RECHAZADO.

La red neuronal se implementa manualmente utilizando únicamente operaciones con numpy. No se utiliza Scikit-Learn en la Raspberry Pi.

Decisión automática sobre cada pieza procesada, controlando tres LEDs:

- Verde para “Aceptado”.

- Amarillo para “Advertencia” o “Revisión”.

- Rojo para “Crítico” o “Rechazado”.

Panel web avanzado desarrollado con Flask, accesible desde cualquier navegador dentro de la misma red.
Este panel muestra:

- Información detallada de cada sensor.

- Valores normalizados usados por la red neuronal.

- Logits y clase predicha por la MLP.

- Historial de piezas recientes.

- Estadísticas acumuladas de producción.

- Mensajes y anuncios del sistema.


Un hilo paralelo dedicado exclusivamente a la adquisición de datos y a la inferencia, garantizando que el panel web siga funcionando sin interrupciones.





## Configuración del hardware

Incluye la configuración de pines GPIO para:

HC-SR04

DHT11

DIP switch

LEDs

In [ ]:
"""
Mini línea de producción inteligente con IA embebida y panel web Flask
Arquitectura MLP: 4 -> 32 -> 3

Panel web:
    - Título: "Proyecto Final - Sistemas Embebidos"
    - Subtítulo: "Profesor: Cesar Hernando Valencia Niño"
    - Espacio para logo de la UTS en la parte superior izquierda
      (colocar archivo en: static/logo_uts.png)

Entradas:
    - Distancia (HC-SR04, cm)
    - Luz normalizada (0-1) [placeholder: fijo 0.6, luego se reemplaza por sensor real]
    - Temperatura (DHT11 o simulada)
    - Modo de operación (0..3) desde DIP switch (2 bits)

Salidas (clases MLP):
    - Clase 0: Producto ACEPTADO (NORMAL)      -> LED verde
    - Clase 1: Producto en REVISIÓN (WARN)     -> LED amarillo
    - Clase 2: Producto RECHAZADO (CRÍTICO)    -> LED rojo
"""

import time
import threading
from datetime import datetime

import numpy as np
from flask import Flask, render_template_string

import RPi.GPIO as GPIO

# Intentar importar DHT11/DHT22
try:
    import Adafruit_DHT
    HAS_DHT = True
except ImportError:
    HAS_DHT = False
    print("⚠️  No se encontró Adafruit_DHT. La temperatura se simulará.")

# ==========================
# 1. CONFIGURACIÓN GPIO
# ==========================
GPIO.setmode(GPIO.BCM)

# HC-SR04
TRIG = 23
ECHO = 24
GPIO.setup(TRIG, GPIO.OUT)
GPIO.setup(ECHO, GPIO.IN)

# DIP switch (2 bits -> modo 0..3)
DIP0 = 5   # bit menos significativo
DIP1 = 6   # bit más significativo
GPIO.setup(DIP0, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
GPIO.setup(DIP1, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)

# LEDs
LED_VERDE  = 17  # Clase 0
LED_AMAR   = 27  # Clase 1
LED_ROJO   = 22  # Clase 2
for p in [LED_VERDE, LED_AMAR, LED_ROJO]:
    GPIO.setup(p, GPIO.OUT)
    GPIO.output(p, 0)

# DHT (ejemplo: DHT11 en GPIO 4)
DHT_SENSOR = Adafruit_DHT.DHT11 if HAS_DHT else None
DHT_PIN = 4

# ==========================
# 2. PESOS DEL MLP 4-32-3
# ==========================
# IMPORTANTE:
#   Reemplaza estos valores de ejemplo por las matrices reales que imprime Colab:
#   W1, b1, W2, b2 con formas:
#       W1: (4, 32)
#       b1: (32,)
#       W2: (32, 3)
#       b2: (3,)

W1 = np.zeros((4, 32), dtype=np.float32)
b1 = np.zeros((32,), dtype=np.float32)
W2 = np.zeros((32, 3), dtype=np.float32)
b2 = np.zeros((3,), dtype=np.float32)

# Ejemplo de cómo deberías pegar lo real (copiado de Colab):
# W1 = np.array([...], dtype=np.float32)
# b1 = np.array([...], dtype=np.float32)
# W2 = np.array([...], dtype=np.float32)
# b2 = np.array([...], dtype=np.float32)


# ==========================
# 3. FUNCIONES DE SENSORES
# ==========================
def medir_distancia_cm(timeout=0.02):
    """Mide distancia en cm con HC-SR04. Devuelve None si falla."""
    GPIO.output(TRIG, 0)
    time.sleep(0.0002)
    GPIO.output(TRIG, 1)
    time.sleep(0.00001)
    GPIO.output(TRIG, 0)

    t0 = time.time()
    while GPIO.input(ECHO) == 0:
        if time.time() - t0 > timeout:
            return None
    start = time.time()

    while GPIO.input(ECHO) == 1:
        if time.time() - start > timeout:
            return None
    end = time.time()

    dur = end - start
    dist_cm = (dur * 34300.0) / 2.0
    return dist_cm


def leer_modo_desde_dip():
    """Lee DIP0 y DIP1 y devuelve modo 0..3."""
    b0 = GPIO.input(DIP0)
    b1 = GPIO.input(DIP1)
    modo = (b1 << 1) | b0
    return modo


def leer_luz_normalizada():
    """
    Placeholder de lectura de luz (0..1).
    - Luego se puede reemplazar con valor de LDR+ADC.
    - Por ahora devolvemos 0.6 como valor de prueba.
    """
    return 0.6


def leer_temperatura_c():
    """Lee temperatura real si hay DHT; de lo contrario, simula ~50°C."""
    if HAS_DHT and DHT_SENSOR is not None:
        for _ in range(5):
            hum, temp = Adafruit_DHT.read_retry(DHT_SENSOR, DHT_PIN)
            if temp is not None:
                return float(temp)
        return 50.0
    else:
        return 50.0


# ==========================
# 4. NORMALIZACIÓN + MLP
# ==========================
def normalizar_entradas(dist_cm, luz_val, temp_c, modo):
    """
    Aplica la misma normalización que en el Colab:
      dist_norm: 4..20 cm -> 0..1
      luz_norm:  0..1
      temp_norm: 20..90 °C -> 0..1
      modo_norm: 0..3 -> 0..1
    """
    dist_clipped = np.clip(dist_cm, 4.0, 20.0)
    dist_norm = (dist_clipped - 4.0) / (20.0 - 4.0)

    luz_norm = float(np.clip(luz_val, 0.0, 1.0))

    temp_clipped = np.clip(temp_c, 20.0, 90.0)
    temp_norm = (temp_clipped - 20.0) / (90.0 - 20.0)

    modo_int = int(np.clip(modo, 0, 3))
    modo_norm = modo_int / 3.0

    x = np.array([dist_norm, luz_norm, temp_norm, modo_norm], dtype=np.float32)
    return x


def mlp_forward(x):
    """
    Forward pass del MLP:
        x: (4,)
        z1 = ReLU(x @ W1 + b1)       -> (32,)
        logits = z1 @ W2 + b2        -> (3,)
    Devuelve: (clase_predicha, logits)
    """
    z1 = np.maximum(0.0, x @ W1 + b1)
    logits = z1 @ W2 + b2
    clase = int(np.argmax(logits))
    return clase, logits


def mostrar_clase_en_leds(clase):
    """Enciende el LED según la clase."""
    if clase == 0:
        GPIO.output(LED_VERDE, 1)
        GPIO.output(LED_AMAR,  0)
        GPIO.output(LED_ROJO,  0)
    elif clase == 1:
        GPIO.output(LED_VERDE, 0)
        GPIO.output(LED_AMAR,  1)
        GPIO.output(LED_ROJO,  0)
    else:
        GPIO.output(LED_VERDE, 0)
        GPIO.output(LED_AMAR,  0)
        GPIO.output(LED_ROJO,  1)


# ==========================
# 5. ESTADO COMPARTIDO
# ==========================
state_lock = threading.Lock()
state = {
    "timestamp": None,
    "dist_cm": None,
    "luz": None,
    "temp_c": None,
    "modo": None,
    "x_norm": None,
    "clase": None,
    "logits": None,
    "count_ok": 0,
    "count_warn": 0,
    "count_crit": 0,
    "history": []  # lista de dicts con últimas N piezas
}
MAX_HISTORY = 20


def actualizar_estado(entry):
    """Actualiza el estado global y la historia."""
    with state_lock:
        state["timestamp"] = entry["timestamp"]
        state["dist_cm"] = entry["dist_cm"]
        state["luz"] = entry["luz"]
        state["temp_c"] = entry["temp_c"]
        state["modo"] = entry["modo"]
        state["x_norm"] = entry["x_norm"]
        state["clase"] = entry["clase"]
        state["logits"] = entry["logits"]

        if entry["clase"] == 0:
            state["count_ok"] += 1
        elif entry["clase"] == 1:
            state["count_warn"] += 1
        else:
            state["count_crit"] += 1

        state["history"].insert(0, entry)
        if len(state["history"]) > MAX_HISTORY:
            state["history"] = state["history"][:MAX_HISTORY]


# ==========================
# 6. HILO DE ADQUISICIÓN
# ==========================
def loop_adquisicion():
    print("▶ Iniciando bucle de adquisición e inferencia MLP...")
    while True:
        dist = medir_distancia_cm()
        if dist is None:
            time.sleep(0.1)
            continue

        luz = leer_luz_normalizada()
        temp = leer_temperatura_c()
        modo = leer_modo_desde_dip()

        x = normalizar_entradas(dist, luz, temp, modo)
        clase, logits = mlp_forward(x)

        mostrar_clase_en_leds(clase)

        ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        estados_txt = ["ACEPTADO (NORMAL)", "EN REVISIÓN (ADVERTENCIA)", "RECHAZADO (CRÍTICO)"]
        estado_str = estados_txt[clase] if 0 <= clase < len(estados_txt) else f"Clase {clase}"

        entry = {
            "timestamp": ts,
            "dist_cm": round(float(dist), 2),
            "luz": round(float(luz), 3),
            "temp_c": round(float(temp), 2),
            "modo": int(modo),
            "x_norm": [float(v) for v in x],
            "clase": clase,
            "estado_str": estado_str,
            "logits": [float(v) for v in logits]
        }
        actualizar_estado(entry)

        print(
            f"[{ts}] Dist={entry['dist_cm']:5.1f} cm | Luz={entry['luz']:.2f} "
            f"| Temp={entry['temp_c']:5.1f} °C | Modo={entry['modo']} "
            f"-> {estado_str} | logits={np.round(logits, 2)}",
            end="\r"
        )

        time.sleep(0.5)


# ==========================
# 7. FLASK – PANEL WEB
# ==========================
app = Flask(__name__)

HTML = """
<!doctype html>
<html lang="es">
<head>
  <meta charset="utf-8">
  <title>Proyecto Final - Sistemas Embebidos</title>
  <meta http-equiv="refresh" content="4">
  <style>
    :root {
      --bg: #0f172a;
      --card: #1f2937;
      --accent: #38bdf8;
      --accent-soft: #0ea5e9;
      --ok: #22c55e;
      --warn: #eab308;
      --crit: #ef4444;
      --text: #e5e7eb;
      --muted: #9ca3af;
    }
    * { box-sizing:border-box; }
    body {
      font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
      background: radial-gradient(circle at top, #1e293b, #020617);
      color: var(--text);
      margin: 0;
      padding: 24px;
    }
    .container {
      max-width: 1200px;
      margin: 0 auto;
    }
    header {
      display:flex;
      align-items:center;
      justify-content:space-between;
      gap:16px;
      margin-bottom: 20px;
    }
    .logo-box {
      width:100px;
      height:100px;
      border-radius:16px;
      background:#020617;
      border:1px solid #1e293b;
      display:flex;
      align-items:center;
      justify-content:center;
      overflow:hidden;
    }
    .logo-box img {
      max-width:90%;
      max-height:90%;
      object-fit:contain;
    }
    .header-text h1 {
      margin:0;
      font-size:1.8rem;
      display:flex;
      align-items:center;
      gap:10px;
    }
    .badge {
      background:rgba(56,189,248,0.15);
      color:var(--accent);
      border-radius:999px;
      padding:2px 10px;
      font-size:0.75rem;
      border:1px solid rgba(56,189,248,0.5);
    }
    .header-text p {
      margin:2px 0;
      color:var(--muted);
      font-size:0.9rem;
    }
    .header-text small {
      color:var(--muted);
    }
    .cards {
      display:grid;
      grid-template-columns: repeat(auto-fit, minmax(260px, 1fr));
      gap:16px;
      margin-bottom:20px;
    }
    .card {
      background: linear-gradient(145deg, #020617, #020617);
      border-radius:14px;
      padding:14px 16px;
      border:1px solid #1e293b;
      box-shadow:0 10px 30px rgba(15,23,42,0.6);
    }
    .card h3 {
      margin:0 0 8px 0;
      font-size:1rem;
      display:flex;
      align-items:center;
      justify-content:space-between;
      gap:8px;
    }
    .card h3 span {
      font-size:0.75rem;
      color:var(--muted);
      font-weight:400;
    }
    .value {
      font-size:1.5rem;
      font-weight:600;
      margin:4px 0;
    }
    .label {
      font-size:0.8rem;
      color:var(--muted);
    }
    .tag {
      display:inline-block;
      padding:2px 8px;
      border-radius:999px;
      font-size:0.7rem;
      border:1px solid #1f2937;
      color:var(--muted);
      margin-top:4px;
    }
    .ok { color:var(--ok); }
    .warn { color:var(--warn); }
    .crit { color:var(--crit); font-weight:600; }

    .grid-two {
      display:grid;
      grid-template-columns: minmax(0, 3fr) minmax(0, 2fr);
      gap:16px;
      margin-bottom:20px;
    }

    .status-pill {
      display:inline-flex;
      align-items:center;
      gap:6px;
      padding:4px 10px;
      border-radius:999px;
      font-size:0.8rem;
      border:1px solid rgba(148,163,184,0.4);
      background:rgba(15,23,42,0.8);
    }

    .history-table {
      width:100%;
      border-collapse:collapse;
      margin-top:10px;
      font-size:0.8rem;
    }
    .history-table th, .history-table td {
      border:1px solid #1f2937;
      padding:6px 4px;
      text-align:center;
    }
    .history-table th {
      background:#020617;
      color:var(--muted);
      font-weight:500;
    }
    .history-table tr:nth-child(even) {
      background:#020617;
    }

    .announcement {
      background: radial-gradient(circle at top left, rgba(56,189,248,0.25), #020617);
      border-radius:14px;
      padding:12px 14px;
      border:1px solid rgba(56,189,248,0.4);
      font-size:0.9rem;
    }
    .announcement h4 {
      margin:0 0 4px 0;
      font-size:0.95rem;
    }
    .announcement p {
      margin:0;
    }

    .footer {
      margin-top:20px;
      text-align:right;
      font-size:0.75rem;
      color:var(--muted);
    }
  </style>
</head>
<body>
  <div class="container">
    <header>
      <div class="logo-box">
        <!-- Colocar el logo de la UTS en: static/logo_uts.png -->
        <img src="/static/logo_uts.png" alt="Logo UTS" onerror="this.style.display='none'">
      </div>
      <div class="header-text">
        <h1>
          Proyecto Final - Sistemas Embebidos
          <span class="badge">Línea de producción inteligente</span>
        </h1>
        <p><b>Profesor:</b> Cesar Hernando Valencia Niño</p>
        <small>Modelo MLP embebido (4 → 32 → 3) ejecutándose en Raspberry Pi</small>
      </div>
    </header>

    <div class="cards">
      <!-- Tarjeta resumen general -->
      <div class="card">
        <h3>Resumen del sistema <span>Estado global</span></h3>
        {% if current %}
          {% if current.clase == 0 %}
            <p class="value ok">ACEPTADO (NORMAL)</p>
          {% elif current.clase == 1 %}
            <p class="value warn">EN REVISIÓN (ADVERTENCIA)</p>
          {% else %}
            <p class="value crit">RECHAZADO (CRÍTICO)</p>
          {% endif %}
          <p class="label">Última actualización: {{ current.timestamp }}</p>
          <p class="label">Vector normalizado x = {{ current.x_norm }}</p>
        {% else %}
          <p class="value">Sin datos aún</p>
          <p class="label">Esperando primeras lecturas de sensores...</p>
        {% endif %}
      </div>

      <!-- Producción -->
      <div class="card">
        <h3>Producción procesada <span>Conteo acumulado</span></h3>
        <p class="label">
          Piezas aceptadas: <span class="ok">{{ count_ok }}</span><br>
          En revisión: <span class="warn">{{ count_warn }}</span><br>
          Rechazadas: <span class="crit">{{ count_crit }}</span>
        </p>
        {% set total = count_ok + count_warn + count_crit %}
        {% if total > 0 %}
          <p class="label">
            Total piezas: {{ total }}<br>
            Rendimiento (OK): {{ (count_ok / total * 100) | round(1) }} %
          </p>
        {% else %}
          <p class="label">Aún no hay piezas procesadas.</p>
        {% endif %}
      </div>

      <!-- Anuncios -->
      <div class="card">
        <h3>Anuncios del sistema <span>Mensajes</span></h3>
        <div class="announcement">
          <h4>Indicaciones generales</h4>
          <p>
            - Verifique las conexiones de los sensores antes de iniciar la prueba.<br>
            - Cambie la posición de la pieza y el modo de operación (DIP) para observar
              cómo la IA modifica la decisión.<br>
            - Use este panel como soporte para el informe final del proyecto.
          </p>
        </div>
      </div>
    </div>

    <!-- Grid con sensores e IA -->
    <div class="grid-two">
      <!-- Sensores -->
      <div class="card">
        <h3>Sensores de la estación de inspección <span>Valores actuales</span></h3>
        {% if current %}
          <div class="cards" style="margin-top:4px;">
            <div class="card">
              <h3>Distancia <span>Posición de la pieza</span></h3>
              <p class="value">{{ current.dist_cm }} <span style="font-size:0.9rem;">cm</span></p>
              <p class="label">Rango recomendado: 8 – 12 cm</p>
              <span class="tag">Entrada 1: dist_norm = {{ current.x_norm[0] | round(3) }}</span>
            </div>
            <div class="card">
              <h3>Luz / Reflectancia <span>Inspección visual</span></h3>
              <p class="value">{{ current.luz }}</p>
              <p class="label">0 = muy oscuro · 1 = muy claro</p>
              <span class="tag">Entrada 2: luz_norm = {{ current.x_norm[1] | round(3) }}</span>
            </div>
            <div class="card">
              <h3>Temperatura <span>Proceso térmico</span></h3>
              <p class="value">{{ current.temp_c }} <span style="font-size:0.9rem;">°C</span></p>
              <p class="label">Rango típico: 40 – 65 °C</p>
              <span class="tag">Entrada 3: temp_norm = {{ current.x_norm[2] | round(3) }}</span>
            </div>
            <div class="card">
              <h3>Modo de operación <span>DIP switch</span></h3>
              <p class="value">{{ current.modo }}</p>
              <p class="label">
                0–3 según combinación de DIP.<br>
                Permite simular diferentes tipos de producto o niveles de exigencia.
              </p>
              <span class="tag">Entrada 4: modo_norm = {{ current.x_norm[3] | round(3) }}</span>
            </div>
          </div>
        {% else %}
          <p class="label">Aún no se han registrado lecturas de sensores.</p>
        {% endif %}
      </div>

      <!-- IA -->
      <div class="card">
        <h3>Salida de la red neuronal <span>Modelo MLP 4 → 32 → 3</span></h3>
        {% if current %}
          <p class="label">Logits (salida sin normalizar de la red):</p>
          <p class="value" style="font-size:1rem;">{{ current.logits }}</p>

          <p class="label">Interpretación de clase:</p>
          {% if current.clase == 0 %}
            <p class="status-pill">
              <span class="ok">●</span> Producto ACEPTADO – condiciones dentro de rango.
            </p>
          {% elif current.clase == 1 %}
            <p class="status-pill">
              <span class="warn">●</span> Producto en REVISIÓN – al menos una variable cercana al límite.
            </p>
          {% else %}
            <p class="status-pill">
              <span class="crit">●</span> Producto RECHAZADO – desviación importante en el proceso.
            </p>
          {% endif %}

          <p class="label" style="margin-top:10px;">
            Nota: la red fue entrenada en Colab con un conjunto de datos sintético que
            simula diferentes condiciones de proceso. Esta implementación solo ejecuta
            la fase de inferencia en la Raspberry Pi.
          </p>
        {% else %}
          <p class="label">Sin inferencia disponible todavía.</p>
        {% endif %}
      </div>
    </div>

    <!-- Historial -->
    <div class="card">
      <h3>Historial reciente de piezas <span>Últimas {{ history|length }} observaciones</span></h3>
      {% if history %}
        <table class="history-table">
          <thead>
            <tr>
              <th>Timestamp</th>
              <th>Dist (cm)</th>
              <th>Luz</th>
              <th>Temp (°C)</th>
              <th>Modo</th>
              <th>Estado</th>
            </tr>
          </thead>
          <tbody>
            {% for row in history %}
              <tr>
                <td>{{ row.timestamp }}</td>
                <td>{{ row.dist_cm }}</td>
                <td>{{ row.luz }}</td>
                <td>{{ row.temp_c }}</td>
                <td>{{ row.modo }}</td>
                {% if row.clase == 0 %}
                  <td class="ok">{{ row.estado_str }}</td>
                {% elif row.clase == 1 %}
                  <td class="warn">{{ row.estado_str }}</td>
                {% else %}
                  <td class="crit">{{ row.estado_str }}</td>
                {% endif %}
              </tr>
            {% endfor %}
          </tbody>
        </table>
      {% else %}
        <p class="label">Aún no hay historial de piezas. Espere a que la línea procese algunos ciclos.</p>
      {% endif %}
    </div>

    <div class="footer">
      Proyecto Final de curso – Sistemas Embebidos · Panel generado con Flask sobre Raspberry Pi
    </div>
  </div>
</body>
</html>
"""

@app.route("/")
def index():
    with state_lock:
        current = {
            "timestamp": state["timestamp"],
            "dist_cm": state["dist_cm"],
            "luz": state["luz"],
            "temp_c": state["temp_c"],
            "modo": state["modo"],
            "x_norm": state["x_norm"],
            "clase": state["clase"],
            "logits": state["logits"],
        } if state["timestamp"] is not None else None

        history = list(state["history"])
        count_ok = state["count_ok"]
        count_warn = state["count_warn"]
        count_crit = state["count_crit"]

    return render_template_string(
        HTML,
        current=current,
        history=history,
        count_ok=count_ok,
        count_warn=count_warn,
        count_crit=count_crit,
    )


# ==========================
# 8. MAIN
# ==========================
if __name__ == "__main__":
    try:
        t = threading.Thread(target=loop_adquisicion, daemon=True)
        t.start()

        print("\nServidor Flask levantándose en http://0.0.0.0:5000 ...")
        app.run(host="0.0.0.0", port=5000, debug=False)

    finally:
        GPIO.output(LED_VERDE, 0)
        GPIO.output(LED_AMAR,  0)
        GPIO.output(LED_ROJO,  0)
        GPIO.cleanup()
